## Keras and Tensorflow model for Aerial Cactus Detection in Kaggle

In [15]:
#Importing tensorflow and keras packages for NN

import tensorflow as tf
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, Dense, Flatten, BatchNormalization, Dropout, LeakyReLU, Flatten
from keras import regularizers
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

#Importing the scikitlearn packages for ML metrics and grid search
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV,KFold
from sklearn.metrics import roc_auc_score


#Importing tqdm for progress bars
from tqdm import tqdm, tqdm_notebook

#Importing pandas and numpy for array manipulation and cv to read in the images as arrays
import numpy as np
import pandas as pd
import cv2 as cv

#For paths
import os

In [3]:
#Setting seed for reproducibility
seed = 12345
np.random.seed(seed)

In [ ]:
#Navigating to directory with data
os.listdir('../input')

In [ ]:
#Setting the path for training and test images
training_path = '../input/train/train/'
test_path = '../input/test/test/'

In [ ]:
#Reading in training labels
train_data = pd.read_csv('../input/train.csv')

In [7]:
train_data.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [8]:
# Convert training images to numpy arrays 

images_train = []
labels_train = []

images = train_data['id'].values
for image_id in tqdm_notebook(images):     #<- tqdm_notebook adds a progress bar in jupyter notebook
    
    image = np.array(cv.imread(training_path + image_id))
    label = train_data[train_data['id'] == image_id]['has_cactus'].values[0]
    
    images_train.append(image)             #<- Add original image
    labels_train.append(label)
    
    # Data Augmentation
    images_train.append(np.flip(image))    #<- Add flipped up down and left-right image
    labels_train.append(label)
    
    images_train.append(np.flipud(image))  #<- Add flipped up down image
    labels_train.append(label)
    
    images_train.append(np.fliplr(image))  #<- Add flipped left right image
    labels_train.append(label)
    
    
images_train = np.asarray(images_train)    #<- Convert combined list to np array
images_train = images_train.astype('float32')
images_train /= 255.                       #<- Normalize          
  
labels_train = np.asarray(labels_train)     

In [9]:
# Convert test images to numpy arrays 

test_images_names = []

for filename in os.listdir(test_path):
    test_images_names.append(filename)
    
test_images_names.sort()

images_test = []

for image_id in tqdm_notebook(test_images_names):
    images_test.append(np.array(cv.imread(test_path + image_id)))
    
images_test = np.asarray(images_test)
images_test = images_test.astype('float32')
images_test /= 255

In [11]:
# Split into test and train using Stratified sampling
x_train, x_valid, y_train, y_valid = train_test_split(images_train, labels_train, test_size = 0.2, stratify = labels_train)

In [12]:
#define AUC as metric since Kaggle competition is evaluated on AUC
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [13]:
#The hyperparameter values being passed to create the model below are the parameters of the best model from a grid search
#The code for the grid search is here. 

def create_model(optimizer='rmsprop', filters_1=16,filters_2=2,filters_3=4,filters_4=32,
                 nn1=300, nn2=100, nn3 = 150, lr=0.01, l2=0.0001, l1=0,
                activation = 'relu', dropout_1=0.4, dropout_2=0, dropout_3=0.2, dropout_4=0.2): 
    
    #Apply l1 L2 regularization to the NN layers
    reg = regularizers.l1_l2(l1=l1, l2=l2)
    
    model = Sequential()
    
    #Our input images are 32*32 pixels and have 3 color channels 
    model.add(Conv2D(filters = filters_1, kernel_size = 3, activation = activation, input_shape = (32, 32, 3)))
    
    model.add(Conv2D(filters = filters_2, kernel_size = 3, activation = activation))
    #Normalizing intermediate layers imrpoves training speed
    model.add(BatchNormalization())
    #Dropout reduces training accuracy but improves test and validation accuracy
    model.add(Dropout(dropout_1))
    
    model.add(Conv2D(filters = filters_3, kernel_size = 1, activation = activation))
    model.add(Conv2D(filters = filters_4, kernel_size = 1, activation = activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_2))
   
    #Output from convolutional layers converted to a flat array 
    model.add(Flatten())
    
    model.add(Dense(nn1, activation = activation,kernel_regularizer=reg))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_3))
    
    model.add(Dense(nn2, activation = activation,kernel_regularizer=reg))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_4))
    
    model.add(Dense(nn3, activation = 'tanh', kernel_regularizer=reg))
    
    #Output layer
    model.add(Dense(1, activation = 'sigmoid'))
    
    #Define optimizer 
    if optimizer =='sgd':
        optimizer = SGD(lr=lr)
    elif optimizer == 'adam':
        optimizer = Adam(lr=lr)
    elif optimizer == 'rmspop':
        optimizer = RMSprop(lr=lr)
        
    #Compile model
    #We use mean_squared_error instead of the more commonly used binary_crossentropy 
    #because it significantly improved the accuracy
    model.compile(optimizer = optimizer , loss= "mean_squared_error", metrics = ['accuracy',auroc])
    

    return model

In [16]:
model = create_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [17]:
# Define where weights will be stored in case we want to use them later.

file_path = 'weights-aerial-cactus.h5'

#Callbacks for training. 
callbacks = [
        #Save the model after every epoch
        ModelCheckpoint(file_path, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max'),
        #Reduce learning rate when loss has stopped improving.
        ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, mode = 'min', min_lr = 0.00001),
        #Stop training when loss has stopped improving by delta
        EarlyStopping(monitor = 'val_loss', min_delta = 1e-10, patience = 15, verbose = 1, restore_best_weights = True)
        ]


In [18]:
#Fitting the model. The epochs and Batch size were decided based on multiple runs.
model.fit(x_train, 
            y_train, 
            batch_size = 128, 
            epochs = 80, 
            validation_data = (x_valid, y_valid),
            verbose = 1    #<- Shows results while running, useful to see progress of fitting
           )

Instructions for updating:
Use tf.cast instead.
Train on 56000 samples, validate on 14000 samples
Epoch 1/80
56000/56000 [==============================] - 180s 3ms/step - loss: 0.1163 - acc: 0.9138 - auroc: 0.9527 - val_loss: 0.3995 - val_acc: 0.5753 - val_auroc: 0.9238
Epoch 2/80
56000/56000 [==============================] - 168s 3ms/step - loss: 0.0734 - acc: 0.9487 - auroc: 0.9843 - val_loss: 0.0913 - val_acc: 0.9238 - val_auroc: 0.9926
Epoch 3/80
56000/56000 [==============================] - 159s 3ms/step - loss: 0.0548 - acc: 0.9638 - auroc: 0.9916 - val_loss: 0.0550 - val_acc: 0.9576 - val_auroc: 0.9982
Epoch 4/80
56000/56000 [==============================] - 165s 3ms/step - loss: 0.0343 - acc: 0.9791 - auroc: 0.9971 - val_loss: 0.0319 - val_acc: 0.9801 - val_auroc: 0.9984
Epoch 5/80
56000/56000 [==============================] - 163s 3ms/step - loss: 0.0278 - acc: 0.9838 - auroc: 0.9980 - val_loss: 0.0266 - val_acc: 0.9839 - val_auroc: 0.9984
Epoch 6/80
56000/56000 [========

56000/56000 [==============================] - 163s 3ms/step - loss: 0.0085 - acc: 0.9935 - auroc: 0.9994 - val_loss: 0.0374 - val_acc: 0.9614 - val_auroc: 0.9973
Epoch 47/80
56000/56000 [==============================] - 174s 3ms/step - loss: 0.0081 - acc: 0.9937 - auroc: 0.9994 - val_loss: 0.0426 - val_acc: 0.9536 - val_auroc: 0.9953
Epoch 48/80
56000/56000 [==============================] - 175s 3ms/step - loss: 0.0080 - acc: 0.9938 - auroc: 0.9996 - val_loss: 0.0643 - val_acc: 0.9309 - val_auroc: 0.9726
Epoch 49/80
56000/56000 [==============================] - 208s 4ms/step - loss: 0.0082 - acc: 0.9932 - auroc: 0.9996 - val_loss: 0.0391 - val_acc: 0.9566 - val_auroc: 0.9884
Epoch 50/80
56000/56000 [==============================] - 189s 3ms/step - loss: 0.0081 - acc: 0.9934 - auroc: 0.9996 - val_loss: 0.0679 - val_acc: 0.9291 - val_auroc: 0.9945
Epoch 51/80
56000/56000 [==============================] - 202s 4ms/step - loss: 0.0078 - acc: 0.9938 - auroc: 0.9995 - val_loss: 0.0432 

In [19]:
#Predict on test data 
predictions = model.predict(images_test, verbose = 1)

4000/4000 [==============================] - 3s 803us/step


In [21]:
#Confusion Matrix 

y_pred_probability = model.predict_proba(x_train)

y_pred = model.predict_classes(x_train)
conf_matrix = confusion_matrix(y_train, y_pred)

In [22]:
conf_matrix

array([[13273,   692],
       [ 1987, 40048]])

In [24]:
#Look at predictions
predictions

array([[0.9923824],
       [0.9980409],
       [0.2532274],
       ...,
       [0.955483 ],
       [0.9977274],
       [0.9979794]], dtype=float32)

In [ ]:
#Generate submission file 

test_df = pd.read_csv('../input/sample_submission.csv')
X_test = []
images_test = test_df['id'].values

for img_id in tqdm_notebook(images_test):
    X_test.append(cv.imread(test_path + img_id))
    
X_test = np.asarray(X_test)
X_test = X_test.astype('float32')
X_test /= 255

y_test_pred = model.predict_proba(X_test)

test_df['has_cactus'] = y_test_pred
test_df.to_csv('aerial-cactus-submission.csv', index = False)